# CatBoost

# Статьи:

https://habr.com/ru/articles/594077/

https://habr.com/ru/companies/otus/articles/778714/

# Документация:
https://catboost.ai/en/docs/concepts/python-reference_catboost_fit

# Импорт data

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np

In [2]:
train_df = pd.read_excel('train.xlsx')
test_df = pd.read_excel('test.xlsx')

In [3]:
train_df.head().style

,ID,Находится в реестре МСП,Размер уставного капитала объявленный,Численность персонала по данным ФНС.Количество,Карточка клиента (внешний источник).Индекс платежной дисциплины Значение,Карточка клиента (внешний источник).Индекс финансового риска Значение,Провозная плата (период 1),Провозная плата (период 2),Объем перевозок(тн) (период 1),Объем перевозок(тн) (период 2),positive_action,Devotion,Churn
0,9,0,0,18,0,27,0,168141,0,50,1,1.000000,0
1,25,0,0,361,0,25,12754399,18211112,31095,34923,0,1.000000,0
2,33,0,443485350,1987,84,21,80731499,105789052,28510,31213,0,48.991020,1
3,86,0,0,0,0,0,0,0,0,0,0,1.000000,0
4,115,0,20773333,1211,96,16,79490420,113617841,30340,41348,2,56.083486,1


In [4]:
X = train_df.drop("Churn", axis=1)
y = train_df["Churn"]

In [5]:
cat_features = list(range(1, 13)) #пропускает первое значение, т.к. ID не катигориальный признак (12 признаков из 13)
print(cat_features)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


In [6]:
print(f"Labels: {set(y)}")
print(f"Zero count: {len(y)-sum(y)}, One count: {sum(y)}")

Labels: {0, 1}
Zero count: 190, One count: 62


In [7]:
# подготовка датасета
train_df.to_csv(('train.tsv'), 
                index=False, sep='\t', header=False)


test_df.to_csv(('test.tsv'), 
              index=False, sep='\t', header=False)


train_df.to_csv(('train.csv'),
                index=False, sep=',', header=True)


test_df.to_csv(('test.csv'),
              index=False, sep=',', header=True)

In [8]:
from catboost.utils import create_cd

feature_names = dict()

for column, name in enumerate(train_df):
    if column == 0:
        continue
    feature_names[column-1] = name
    

create_cd(
    
    cat_features=list(range(1, train_df.columns.shape[0])),
    feature_names=feature_names,
    output_path='train.cd'
)

In [9]:
pool1 = Pool(data=X, label=y, cat_features=cat_features)

pool2 = Pool(
    data=os.path.join(dataset_dir, 'train.csv'),
    delimiter=',',
    column_description=os.path.join(dataset_dir, 'train.cd'),
    has_header=True
)

pool3 = Pool(data=X, cat_features=cat_features)

X_prepared = X.values.astype(str).astype(object)

pool4 = Pool(
    data=FeaturesData(
        cat_feature_data=X_prepared,
        cat_feature_names=list(X)
    ),
    label=y.values
)

print("Dataset shape")
print(f"Dataset 1: {str(pool1.shape)}")
print(f"Dataset 2: {str(pool2.shape)}")
print(f"Dataset 3: {str(pool3.shape)}")
print(f"Dataset 4: {str(pool4.shape)}")
print()
print("Column names")
print(f"Dataset 1: {pool1.get_feature_names()}")
print(f"Dataset 2: {pool2.get_feature_names()}")
print(f"Dataset 3: {pool3.get_feature_names()}")
print(f"Dataset 4: {pool4.get_feature_names()}")

NameError: name 'Pool' is not defined

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.45, random_state=42)

In [ ]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
    iterations=200,
    learning_rate=0.15
)

In [ ]:
model.fit(X_train, y_train,
         cat_features=cat_features,
         eval_set=(X_test, y_test),
          verbose=False
)

print(f"Model is fitted: {str(model.is_fitted())}")
print(f"Model params: {model.get_params()}")

In [ ]:
Model is fitted: True
Model params: {'iterations': 200, 'learning_rate': 0.15}

In [ ]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
    iterations=200,
    learning_rate=0.15
)

model.fit(X_train, y_train,
         cat_features=cat_features,
         eval_set=(X_test, y_test),
         verbose=35
)

In [ ]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
    iterations=200,
    random_seed=63,
    learning_rate=0.15
    custom_loss=['AUC', 'Accuracy']
)

model.fit(X_train, y_train,
         cat_features=cat_features,
         eval_set=(X_test, y_test),
          verbose=False,
          plot=True
)

# Нормализация Log Transformation

In [46]:
df[['Размер уставного капитала объявленный', 'Численность персонала по данным ФНС.Количество', 
    'Карточка клиента (внешний источник).Индекс платежной дисциплины Значение', 
    'Карточка клиента (внешний источник).Индекс финансового риска Значение',
    'Провозная плата (период 1)', 'Провозная плата (период 2)',
    'Объем перевозок(тн) (период 1)', 'Объем перевозок(тн) (период 2)',
    'positive_action', 'Devotion' 
    ]] = np.log1p(df[['Размер уставного капитала объявленный', 'Численность персонала по данным ФНС.Количество', 
    'Карточка клиента (внешний источник).Индекс платежной дисциплины Значение', 
    'Карточка клиента (внешний источник).Индекс финансового риска Значение',
    'Провозная плата (период 1)', 'Провозная плата (период 2)',
    'Объем перевозок(тн) (период 1)', 'Объем перевозок(тн) (период 2)',
    'positive_action', 'Devotion']])

# категории для обучения и предсказания

In [17]:
y = df['Devotion'] # используется параметр с плавающей точкой

X = df[['Размер уставного капитала объявленный', 'Численность персонала по данным ФНС.Количество', 
    'Карточка клиента (внешний источник).Индекс платежной дисциплины Значение', 
    'Карточка клиента (внешний источник).Индекс финансового риска Значение',
    'Провозная плата (период 1)', 'Провозная плата (период 2)',
    'Объем перевозок(тн) (период 1)', 'Объем перевозок(тн) (период 2)',
    'positive_action']]
